In [1]:
import requests
import pandas as pd
import sqlite3
import lxml.html

In [2]:
r = requests.get("https://www.hockey-reference.com/leagues/NHL_2019_skaters.html")

In [3]:
html                = lxml.html.fromstring(r.text)
stats_table         = html.get_element_by_id("stats")
tablebody           = stats_table.getchildren()[3]

In [6]:
tablebody.getchildren()[0].getchildren()[1].text_content()


'Justin Abdelkader'

In [7]:
data = {
        "player":    [],
        "age":       [],
        "team":      [],
        "pos":       [],
        "gp":        [],
        "goals":     [],
        "assists":   [],
        "points":    [],
        "plus/minus":[],
        "pims":      []
     
    
}

for row in tablebody.getchildren():
    if row.getchildren()[0].text_content() == "Rk":
        continue
    data["player"].append(row.getchildren()[1].text_content())
    data["age"].append(row.getchildren()[2].text_content())
    data["team"].append(row.getchildren()[3].text_content())
    data["pos"].append(row.getchildren()[4].text_content())
    data["gp"].append(row.getchildren()[5].text_content())
    data["goals"].append(row.getchildren()[6].text_content())
    data["assists"].append(row.getchildren()[7].text_content())
    data["points"].append(row.getchildren()[8].text_content())
    data["plus/minus"].append(row.getchildren()[9].text_content())
    data["pims"].append(row.getchildren()[10].text_content())

df = pd.DataFrame(data)
df

,player,age,team,pos,gp,goals,assists,points,plus/minus,pims
0,Justin Abdelkader,31,DET,LW,34,5,6,11,0,26
1,Pontus Aberg,25,ANA,LW,28,10,7,17,0,12
2,Noel Acciari,27,BOS,C,26,0,1,1,-6,15
3,Kenny Agostino,26,MTL,LW,18,1,8,9,2,4
4,Sebastian Aho,21,CAR,RW,31,11,22,33,4,12
5,Mark Alt,27,COL,D,2,0,0,0,0,0
6,Karl Alzner,30,MTL,D,8,0,1,1,2,2
7,Michael Amadio,22,LAK,C,23,1,2,3,-5,4
8,Joey Anderson,20,NJD,RW,11,1,1,2,-2,4
9,Josh Anderson,24,CBJ,RW,32,13,3,16,15,27


In [8]:
db = sqlite3.connect(":memory:")
cursor = db.cursor()
cursor.execute("""
        CREATE TABLE NHL_PLAYERS(PLAYER,
        AGE,
        TEAM,
        POS,
        GP,
        GOALS,
        ASSISTS,
        POINTS,
        PLUS_MINUS,
        PIMS)
""")

for row in df.itertuples():
    insert_sql_statement = """
        INSERT INTO NHL_PLAYERS(PLAYER,AGE,TEAM,POS,GP,GOALS,ASSISTS,POINTS,PLUS_MINUS,PIMS)
        VALUES (?,?,?,?,?,?,?,?,?,?)   
    """
    cursor.execute(insert_sql_statement, row[1:])
db.commit()
#db.close()

# Checking for Carolina Players

In [9]:
for row in cursor.execute("""
      SELECT *
      FROM NHL_PLAYERS
      WHERE TEAM = "CAR"
"""):
    print(row)

('Sebastian Aho', '21', 'CAR', 'RW', '31', '11', '22', '33', '4', '12')
('Jake Bean', '20', 'CAR', 'D', '2', '0', '0', '0', '0', '2')
('Clark Bishop', '22', 'CAR', 'C', '14', '1', '1', '2', '-3', '4')
('Trevor Carrick', '24', 'CAR', 'D', '1', '0', '0', '0', '-1', '5')
('Calvin de Haan', '27', 'CAR', 'D', '31', '1', '4', '5', '4', '14')
('Phillip Di Giuseppe', '25', 'CAR', 'LW', '18', '1', '3', '4', '-2', '8')
('Justin Faulk', '26', 'CAR', 'D', '31', '1', '8', '9', '-2', '23')
('Micheal Ferland', '26', 'CAR', 'LW', '25', '11', '4', '15', '4', '13')
('Haydn Fleury', '22', 'CAR', 'D', '10', '0', '1', '1', '1', '0')
('Warren Foegele', '22', 'CAR', 'LW', '31', '3', '2', '5', '-15', '10')
('Dougie Hamilton', '25', 'CAR', 'D', '31', '3', '7', '10', '-11', '18')
('Janne Kuokkanen', '20', 'CAR', 'LW', '2', '0', '0', '0', '0', '2')
('Saku Maenalanen', '24', 'CAR', 'RW', '1', '0', '0', '0', '0', '0')
('Jordan Martinook', '26', 'CAR', 'LW', '31', '8', '3', '11', '-1', '12')
('Brock McGinn', '24', 

# Players who have scored more than 20 goals
## have to cast as all fields are scored as text.

In [10]:
for row in cursor.execute("""
      SELECT *
      FROM NHL_PLAYERS
      WHERE GOALS >= CAST(20 as decimal)
"""):
    print(row)

('Patrik Laine', '20', 'WPG', 'RW', '32', '23', '6', '29', '-4', '20')
('Gabriel Landeskog', '26', 'COL', 'LW', '33', '21', '19', '40', '21', '22')
('Nathan MacKinnon', '23', 'COL', 'C', '33', '21', '31', '52', '16', '20')
('Sean Monahan', '24', 'CGY', 'C', '33', '20', '18', '38', '7', '4')
('Alex Ovechkin', '33', 'WSH', 'LW', '32', '29', '14', '43', '14', '14')
('David Pastrnak', '22', 'BOS', 'RW', '32', '21', '18', '39', '4', '16')
('Joe Pavelski', '34', 'SJS', 'C', '33', '20', '7', '27', '1', '12')
('Brayden Point', '22', 'TBL', 'C', '33', '21', '20', '41', '9', '14')
('Mark Scheifele', '25', 'WPG', 'C', '32', '20', '23', '43', '15', '22')
('Jeff Skinner', '26', 'BUF', 'LW', '33', '22', '11', '33', '19', '12')
('John Tavares', '28', 'TOR', 'C', '33', '20', '14', '34', '6', '12')
